In [5]:
import requests, codecs, os, datetime, json, sys

In [2]:
def fetch_search_results(section):
    base = 'https://philadelphia.craigslist.org/search/' + section + '?format=rss'
    resp = requests.get(base, timeout=3)
    resp.raise_for_status()  # <- no-op if status==200
    return resp.content

In [9]:
def write_results(result, section):
    now = datetime.datetime.now()
    dest = "raw/" + section + now.strftime('/%Y-%m-%d/%H_%M.xml')
    try:
        os.makedirs(os.path.dirname(dest))
    except:
        pass
    
    try:
        f = codecs.open(dest,'w')
        f.write(result)
    except:
        print ("Error saving XML for section %s: $s. Continuing..." % section, sys.exc_info()[0])
        pass

In [10]:
try:
    data = open('sections.json').read()
    sections = json.loads(data)
except:
    raise Exception("Error reading craigslist sections JSON: %s" % sys.exc_info()[0])

In [ ]:
for section in sections.values():
    result = fetch_search_results(section)
    write_results(result, section)